# I394E managed lanes.

In this notebook we will run through:

1. Using a configuration file to run lasso  
2. Setting up a base scenario and applying projects  
3. Transforming the standard network format to the MetCouncil expected format    
4. Exporting the network to a shapefile and csvs  

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit

In [2]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

## Read a Config File

Let's examine the configuration file and store it as `my_config` variable.  

Configuration files are written in YAML and read in as python dictionaries.

In [4]:
MY_CONFIG_FILE = "C:/projects/met_council_git/data/interim/project_cards/settings/i394_managed_lanes.yaml"

with open(MY_CONFIG_FILE) as f:
        my_config = yaml.safe_load(f)
        
import json
print(json.dumps(my_config, indent=2))

## Alternatively this could be written in the notebook our selected via a notebook GUI

{
  "base_scenario": {
    "input_dir": "C:/projects/met_council_git/data/interim/networks/base_network",
    "shape_file_name": "shape.geojson",
    "link_file_name": "link.json",
    "node_file_name": "node.geojson",
    "transit_directory": "",
    "validate_network": false
  },
  "scenario": {
    "project_cards_filenames": [
      "C:/projects/met_council_git/data/interim/project_cards/i394_reversible_managed_lane.yml"
    ],
    "card_directory": [],
    "glob_search": [],
    "tags": []
  },
  "output": {
    "format": "geojson",
    "directory": "",
    "prefix": ""
  }
}


## Create a Base Scenario

Base scenarios must at the least specify a highway network but can also specify a directory where transit networks can be found.  

In this step the highway and transit networks are read in and validated to each other.  

In some cases, you may want to override the validation (after reviewing the errors) using the flag: `validate = False`.

In [5]:
base_wrangler_path = os.path.join(os.path.dirname((os.path.dirname(os.path.abspath('')))),"network_wrangler")
WranglerLogger.info("Base Wrangler Path: {}".format(base_wrangler_path))

base_scenario = Scenario.create_base_scenario(
        my_config["base_scenario"]["shape_file_name"],
        my_config["base_scenario"]["link_file_name"],
        my_config["base_scenario"]["node_file_name"],
        roadway_dir=my_config["base_scenario"]["input_dir"],
        transit_dir=my_config["base_scenario"]["input_dir"],
        validate = False
    )

2020-05-27 11:32:59, INFO: Base Wrangler Path: D:\network_wrangler
2020-05-27 11:32:59, INFO: Reading from following files:
-C:/projects/met_council_git/data/interim/networks/base_network\link.json
-C:/projects/met_council_git/data/interim/networks/base_network\node.geojson
-C:/projects/met_council_git/data/interim/networks/base_network\shape.geojson.
2020-05-27 11:38:04, INFO: Read 1134074 links from C:/projects/met_council_git/data/interim/networks/base_network\link.json
2020-05-27 11:38:04, INFO: Read 354346 nodes from C:/projects/met_council_git/data/interim/networks/base_network\node.geojson
2020-05-27 11:38:04, INFO: Read 1134074 shapes from C:/projects/met_council_git/data/interim/networks/base_network\shape.geojson
2020-05-27 11:45:42, INFO: Read in transit feed from: C:/projects/met_council_git/data/interim/networks/base_network
2020-05-27 11:45:42, DEBUG: ...agency.txt:
  agency_id       agency_name                   agency_url  agency_timezone  \
0         0     Metro Transi

2020-05-27 11:45:42, DEBUG: ...frequencies.txt:
                             trip_id  headway_secs  start_time  end_time
0  14940701-JUN19-MVS-BUS-Weekday-01          3600     21600.0   32400.0
1  14941148-JUN19-MVS-BUS-Weekday-01           830     21600.0   32400.0
2  14941151-JUN19-MVS-BUS-Weekday-01           540     21600.0   32400.0
3  14941153-JUN19-MVS-BUS-Weekday-01           696     32400.0   54000.0
4  14941163-JUN19-MVS-BUS-Weekday-01           830     32400.0   54000.0
5  14941643-JUN19-MVS-BUS-Weekday-01           900     21600.0   32400.0
6  14941652-JUN19-MVS-BUS-Weekday-01          1200     32400.0   54000.0
7  14942968-JUN19-MVS-BUS-Weekday-01          5400     21600.0   32400.0
8  14943414-JUN19-MVS-BUS-Weekday-01          2160     21600.0   32400.0
9  14943415-JUN19-MVS-BUS-Weekday-01          5400     21600.0   32400.0
2020-05-27 11:45:42, DEBUG: ...shapes.txt:
  shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence shape_osm_node_id  \
0  4520004     44.923257   

2020-05-27 11:45:42, DEBUG: starting ox_graph()
2020-05-27 11:45:42, DEBUG: GRAPH NODES: Index(['model_node_id', 'osm_node_id', 'shstReferenceId', 'drive_node',
       'walk_node', 'bike_node', 'bus_only', 'rail_only', 'geometry', 'x',
       'y'],
      dtype='object')
2020-05-27 11:45:45, DEBUG: starting ox.gdfs_to_graph()
2020-05-27 11:54:14, DEBUG: finished ox.gdfs_to_graph()


#### Create project cards from projects that are explicitely specified in config


In [6]:
if len(my_config["scenario"]["project_cards_filenames"]) > 0:
    project_cards_list = [
        ProjectCard.read(filename, validate=False)
        for filename in my_config["scenario"]["project_cards_filenames"]
    ]
else: 
    project_cards_list = []
project_cards_list


## Create Scenario

A scenario is constructed with a base scenario and then selecting project cards to be added to that base scenario to create the new scenario.

Projects can be added a variety of ways:

 1. `card_directory` + `tags` will search a directory add project's who's project tags match *at least one of* the tags in the keyword.
 2. `card_directory` + `glob_search` will search a directory add project's who's file name matches the [glob search text](https://docs.python.org/3/library/glob.html)
 3. `project_cards_list` is a list of ProjectCard objects
 
Optionally, you may specify that project card formats are not validated by setting they keyword: 
   `validate = False`
   
Projects that are not added in the initial scenario development can be added by using the following methods:  

 - `add_project_card_from_file()`  
 - `add_project_cards_from_directory()`  
 - `add_project_cards_from_tags`
 
Or by directly adding the project to the scenario's project attribute by running:

```python
my_project = ProjectCard.read(path_to_card)
my_scenario.projects += my_project

```


In [7]:
my_scenario=None

my_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    project_cards_list=project_cards_list,
    validate_project_cards=False,
)

2020-05-27 11:54:14, INFO: Creating Scenario
2020-05-27 11:54:14, DEBUG: Adding project cards from List.
I394E Reversible Lanes


### Apply all projects in scenario

In [8]:
WranglerLogger.info("\nProjects in queue to be applied: \n - {}".format("\n - ".join(my_scenario.get_project_names())))
WranglerLogger.info("\n[Before] Applied Projects: \n - {}".format("\n - ".join(my_scenario.applied_projects)))

my_scenario.apply_all_projects()

WranglerLogger.info("\n[After] Applied Projects: \n - {}".format("\n - ".join(my_scenario.applied_projects)))

2020-05-27 11:57:22, INFO: 
Projects in queue to be applied: 
 - I394E Reversible Lanes
2020-05-27 11:57:22, INFO: 
[Before] Applied Projects: 
 - 
2020-05-27 11:57:22, DEBUG: Ordered Project Cards: {'i394e reversible lanes': <network_wrangler.projectcard.ProjectCard object at 0x000001CE88F2B188>}
2020-05-27 11:57:22, DEBUG: Project Cards: [<network_wrangler.projectcard.ProjectCard object at 0x000001CE88F2B188>]
2020-05-27 11:57:22, INFO: Applying I394E Reversible Lanes
2020-05-27 11:57:22, INFO: Applying Project to Roadway Network: I394E Reversible Lanes
2020-05-27 11:57:22, DEBUG: validating selection
2020-05-27 11:57:22, DEBUG: Selection Key: ((model_link_id==394063 or model_link_id==404192 or model_link_id==403006 or model_link_id==402701 or model_link_id==402700 or model_link_id==403289 or model_link_id==382272 or model_link_id==419239 or model_link_id==419238))
2020-05-27 11:57:22, DEBUG: Building selection query
2020-05-27 11:57:22, DEBUG: Selecting features:
((model_link_id==39

d:\network_wrangler\network_wrangler\roadwaynetwork.py:847: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_links["i"] = 0


2020-05-27 11:57:23, INFO: 
[After] Applied Projects: 
 - I394E Reversible Lanes


In [9]:
#my_scenario.road_net.links_df.info()

In [10]:
#my_scenario.road_net.links_df[my_scenario.road_net.links_df["segment_id"] == 5]

# Write out as MetCouncil Model Roadway Network
Everything above was done in "pure wrangler" rather than lasso.  However, we will need Lasso in order to add the MetCouncil specific variables. You can create a lasso ModelRoadwayNetwork object from the roadway network object and feed it any additional parameters from that `my_config` variable.

You can see that the link variables for this network are the same as the standard roadway network at this point but that will change.

Since this is a GeoDataFrame you can also use build-in Geopandas features to make simple plots based on these variables.

In [11]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)

2020-05-27 12:03:43, INFO: Lasso base directory set as: D:\lasso


In [12]:
#model_road_net.links_df.info()

In [13]:
#model_road_net.links_df.loc[model_road_net.links_df["HOV"] == "NaN", "HOV"] = "0"

In [14]:
#model_road_net.links_df[model_road_net.links_df["HOV"] != "0"]

In [15]:
#model_road_net.nodes_df.info()

## Add MetCouncil variables
At this point, we need to calculate all the variables into what MetCouncil's model is expecting. The method `roadway_standard_to_met_council_network()` broadly does the following:  
 
- creates a parallel managed lane network
- calculates additional variables based on geography or other variables (i.e. county, assignment group, area type, etc)
- flattens variables stored as continuous time values and determines their value by time period (i.e. lanes_am)   
- reprojects into MetCouncil's projection

In [16]:
model_road_net.roadway_standard_to_met_council_network()

2020-05-27 12:03:43, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-05-27 12:03:43, INFO: Creating managed lane network.
2020-05-27 12:03:43, INFO: Creating network with duplicated managed lanes


d:\network_wrangler\network_wrangler\roadwaynetwork.py:1768: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_links_df[attr] = ""
d:\network_wrangler\network_wrangler\roadwaynetwork.py:1762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_links_df[gp_attr] = ml_links_df[attr]
C:\Anaconda3\envs\lasso_env\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior

2020-05-27 12:04:18, INFO: Creating calculated roadway variables.
2020-05-27 12:04:18, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type
2020-05-27 12:04:29, DEBUG: Reading Area Type Shapefile D:\lasso\metcouncil_data\area_type\ThriveMSP2040CommunityDesignation.shp


C:\Anaconda3\envs\lasso_env\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\Anaconda3\envs\lasso_env\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(4326 != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)


2020-05-27 12:06:27, DEBUG: Area Type Codes Used: {23: 4, 24: 3, 25: 2, 35: 2, 36: 1, 41: 1, 51: 1, 52: 1, 53: 1, 60: 1}
2020-05-27 12:06:27, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-05-27 12:06:30, INFO: Adding roadway network variable for county using a spatial join with: D:\lasso\metcouncil_data\county\cb_2017_us_county_5m.shp


...

d:\lasso\lasso\roadway.py:934: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].astype(network_var_type)


2020-05-27 12:11:56, INFO: Finished adding counts variable: AADT
2020-05-27 12:11:56, INFO: Finished creating ML lanes variable: ML_lanes
2020-05-27 12:11:56, INFO: Hov corridor Variable 'segment_id' already in network. Returning without overwriting.
2020-05-27 12:11:56, INFO: Managed Variable 'managed' already in network. Returning without overwriting.
2020-05-27 12:11:56, INFO: Overwriting existing distance Variable 'distance' already in network


C:\Anaconda3\envs\lasso_env\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


2020-05-27 12:12:49, INFO: Calculating distance for centroid connectors
2020-05-27 12:12:56, INFO: Filling nan for network from network wrangler
2020-05-27 12:13:38, INFO: Converting variable type to MetCouncil standard
2020-05-27 12:13:38, INFO: Splitting variables by time period and category
2020-05-27 12:14:30, INFO: Setting Coordinate Reference System to EPSG 26915


C:\Anaconda3\envs\lasso_env\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\Anaconda3\envs\lasso_env\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\Anaconda3\envs\lasso_env\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [17]:
#model_road_net.links_metcouncil_df.query('lanes == 0 & drive_access == 1 & centroidconnect')

In [18]:
#pd.options.display.max_columns = None
#model_road_net.links_metcouncil_df[model_road_net.links_metcouncil_df["model_link_id"] == 1390975]["segment_id"]


In [19]:
#model_road_net.links_metcouncil_df[model_road_net.links_metcouncil_df.model_link_id.isin([390975])]

In [20]:
#model_road_net.links_metcouncil_df.segment_id.value_counts()
pd.crosstab(model_road_net.links_metcouncil_df.segment_id, model_road_net.links_metcouncil_df.managed)

managed,0,1
segment_id,,
0,1134083,0
5,9,9


In [21]:
#model_road_net.links_metcouncil_df.plot("lanes_AM")

## Export to shapefile

As a last step, the network can be exported to a shapefile and paired CSVs after removing extraneous variables.

(note that this step will also run the `roadway_standard_to_met_council_network()` method but I wanted to show it to you piecewise)

In [22]:
model_road_net.write_roadway_as_shp(
    output_link_shp = "C:/projects/met_council_git/data/interim/networks/fixed_width/link.shp",
    output_node_shp = "C:/projects/met_council_git/data/interim/networks/fixed_width/node.shp"
)

2020-05-27 12:17:01, INFO: Writing Network as Shapefile
2020-05-27 12:17:01, DEBUG: Output Variables: 
 - model_link_id
 - link_id
 - A
 - B
 - shstGeometryId
 - distance
 - roadway
 - name
 - roadway_class
 - bike_access
 - walk_access
 - drive_access
 - truck_access
 - trn_priority_AM
 - trn_priority_MD
 - trn_priority_PM
 - trn_priority_NT
 - ttime_assert_AM
 - ttime_assert_MD
 - ttime_assert_PM
 - ttime_assert_NT
 - lanes_AM
 - lanes_MD
 - lanes_PM
 - lanes_NT
 - price_sov_AM
 - price_hov2_AM
 - price_hov3_AM
 - price_truck_AM
 - price_sov_MD
 - price_hov2_MD
 - price_hov3_MD
 - price_truck_MD
 - price_sov_PM
 - price_hov2_PM
 - price_hov3_PM
 - price_truck_PM
 - price_sov_NT
 - price_hov2_NT
 - price_hov3_NT
 - price_truck_NT
 - roadway_class_idx
 - assign_group
 - access_AM
 - access_MD
 - access_PM
 - access_NT
 - mpo
 - area_type
 - county
 - centroidconnect
 - AADT
 - count_year
 - count_AM
 - count_MD
 - count_PM
 - count_NT
 - count_daily
 - model_node_id
 - N
 - osm_node_id

# Export to fixed width file

In [23]:
model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/link.txt",
    output_node_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/node.txt",
    output_link_header_width_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/link_header.csv",
    output_node_header_width_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/node_header.csv",
    output_cube_network_script = "C:/projects/met_council_git/data/interim/networks/fixed_width/make_complete_network_from_fixed_width_file.s"
)

2020-05-27 12:23:22, DEBUG: Network Link Variables: 
 - A
 - B
 - HOV
 - access
 - bike_access
 - bike_facility
 - bridge
 - bus_only
 - distance
 - drive_access
 - egress
 - geometry
 - lanes
 - locationReferences
 - managed
 - max_speed
 - model_link_id
 - name
 - osm_link_id
 - price
 - rail_only
 - ref
 - roadway
 - segment_id
 - shstGeometryId
 - shstReferenceId
 - trn_priority
 - truck_access
 - ttime_assert
 - tunnel
 - u
 - v
 - walk_access
 - width
 - area_type
 - county
 - centroidconnect
 - mpo
 - assign_group
 - roadway_class
 - AADT
 - count_AM
 - count_MD
 - count_PM
 - count_NT
 - count_daily
 - count_year
 - ML_lanes
 - trn_priority_AM
 - trn_priority_MD
 - trn_priority_PM
 - trn_priority_NT
 - ttime_assert_AM
 - ttime_assert_MD
 - ttime_assert_PM
 - ttime_assert_NT
 - lanes_AM
 - lanes_MD
 - lanes_PM
 - lanes_NT
 - ML_lanes_AM
 - ML_lanes_MD
 - ML_lanes_PM
 - ML_lanes_NT
 - price_sov_AM
 - price_hov2_AM
 - price_hov3_AM
 - price_truck_AM
 - price_sov_MD
 - price_hov2_M

# Write out as MetCouncil Model Transit Network

Similar to the roadway network, the first step is to convert it to a Lasso object, and then write it to a cube line file.  Optionally, you could also export it to a shapefile to inspect using other means. 

In [24]:
#standard_transit = StandardTransit.fromTransitNetwork(my_scenario.transit_net)
#standard_transit.feed

Write out the StandardTransit Lasso object to a cube line file:

In [25]:
#standard_transit.write_as_cube_lin()